In [29]:
import bpy
import numpy as np
import pickle as pkl


planets = ['Sun','Jupiter','Saturn','Uranus','Neptune','Pluto']

In [3]:
f = open('v_out.pkl', 'rb')

v_out = pkl.load(f)
f.close()
f = open('x_out.pkl', 'rb')

x_out = pkl.load(f)
f.close()

In [5]:
print(x_out.shape)

(365, 6, 3)


In [79]:
def add_ordbit(objectName:str='Jupiter'):
    
    assert objectName in planets, 'That is not a planet'
    p_idx = planets.index(objectName)
    
    obj = bpy.data.objects[objectName]
#     obj.rigid_body.enabled = True
    obj.animation_data_clear()
    
    n_frames = 365
    bpy.context.scene.frame_start = 1
    bpy.context.scene.frame_end = n_frames
    
    for frame in range(n_frames):
        
        bpy.context.scene.frame_set(frame)
        x,y,z = x_out[frame,p_idx,:]
        vx,vy,vz = v_out[frame,p_idx,:] # one is for jupiter
        obj.location.x = x*5
        obj.location.y = y*5
        obj.location.z = z*5
#         obj. = (vx, vy, vz)
        obj.keyframe_insert(data_path="location")

def add_trail(objectName:str='Jupiter'):
    
    # Get a reference to the object
    obj = bpy.data.objects[objectName]
#     print(obj)
    # Add a particle system to the object
    particle_system = obj.modifiers.new(type='PARTICLE_SYSTEM', name=objectName + '_trail')
    
#     print(particle_system.particle_system)
    
    # Set the particle system settings
    settings = particle_system.particle_system.settings
    settings.count = 1000  # Number of particles
    settings.lifetime = 365  # Lifetime of the particles in frames
    settings.frame_start = 1  # Start frame of the particle system
    settings.frame_end = 365  # End frame of the particle system
    settings.emit_from = 'VERT'  # Emit particles from the vertices of the object
    settings.use_modifier_stack = True  # Use the object's modifier stack to determine particle emission

    # Add a particle instance modifier to the object
    particle_instance = obj.modifiers.new(type='PARTICLE_INSTANCE', name='Trail Instance')
    print(dir(particle_instance))
    # Set the particle instance settings
    particle_instance.object = bpy.data.objects[objectName + '_trail']  # Object to use as the particle instance
    particle_instance.use_parent_particle = True  # Use the parent particle system to determine particle emission

In [80]:
for planet in planets:
    add_ordbit(planet)
    add_trail(planet)


['__doc__', '__module__', '__slots__', 'axis', 'bl_rna', 'index_layer_name', 'is_active', 'is_override_data', 'name', 'object', 'particle_amount', 'particle_offset', 'particle_system', 'particle_system_index', 'position', 'random_position', 'random_rotation', 'rna_type', 'rotation', 'show_alive', 'show_dead', 'show_expanded', 'show_in_editmode', 'show_on_cage', 'show_render', 'show_unborn', 'show_viewport', 'space', 'type', 'use_apply_on_spline', 'use_children', 'use_normal', 'use_path', 'use_preserve_shape', 'use_size', 'value_layer_name']


KeyError: 'bpy_prop_collection[key]: key "Sun_trail" not found'

In [ ]:
# Variable for currently active object
myobj = bpy.context.object['Earth']
# Alternatively, if you know that the object is called 'Cube'
# you can reach it by
# myobj = bpy.data.objects['Cube']

# Clear all previous animation data
myobj.animation_data_clear()

# set first and last frame index
total_time = 2*pi # Animation should be 2*pi seconds long
fps = 24 # Frames per second (fps)
bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = round(total_time*fps)+1

# loop of frames and insert keyframes at every frame
nlast = bpy.context.scene.frame_end
for n in range(nlast):
    t = total_time*n/nlast

    # Set frame like this
    bpy.context.scene.frame_set(n)
    
    # 𝐫(𝑡)=[5cos(𝑡),5sin(𝑡),sin(5𝑡)]
    
    # Set current location like this
    myobj.location.x = 5*cos(t)
    myobj.location.y = 5*sin(t)
    myobj.location.z = 5*sin(t)
    # myobj.location.x = ...
    # myobj.location.y = ...
    # myobj.location.z = ...

    # Insert new keyframe for "location" like this
    myobj.keyframe_insert(data_path="location")